In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

## LOB Cleaning

In [22]:
# insert date name
name = '2024-06-26'

lob = pd.read_table(f"./data/UoB_Set01_{name}LOBs.txt")
# tape = pd.read_csv("UoB_Set01_2024-06-24tapes.csv",
                 # names=['Timestamp', 'Price', 'Quantity'])

In [23]:
lob

,"[0.000, Exch0, [['bid', []], ['ask', []]]]"
0,"[1.922, Exch0, [['bid', []], ['ask', [[708, 4]..."
1,"[2.790, Exch0, [['bid', [[5, 1]]], ['ask', [[7..."
2,"[3.255, Exch0, [['bid', [[59, 1]]], ['ask', [[..."
3,"[3.379, Exch0, [['bid', [[59, 1]]], ['ask', [[..."
4,"[3.441, Exch0, [['bid', [[108, 1], [59, 1]]], ..."
...,...
320093,"[30599.728, Exch0, [['bid', [[105, 9], [104, 8..."
320094,"[30599.759, Exch0, [['bid', [[105, 9], [104, 8..."
320095,"[30599.821, Exch0, [['bid', [[105, 9], [104, 8..."
320096,"[30599.914, Exch0, [['bid', [[105, 9], [104, 8..."


In [24]:
lob.isnull().sum()

[0.000, Exch0, [['bid', []], ['ask', []]]]    0
dtype: int64

There is no missing data in this LOB (24/06/2024).

First, we extract the timestamp information using regex...

In [25]:
pattern = r'(\d+.+?),'
timestamps = lob.iloc[:,0].str.extract(pattern)
lob['Timestamps'] = timestamps

We then extract the ask data in a similar way. However, we need to use two regex statements and then converge them to one list, to account for the cases where there is only one price in the ask section.

In [26]:
pattern1 = r"'ask', (\[\[(?:\d+,\s*\d+)\]\])"
pattern2 = r"'ask', (\[(\[(?:\d+,\s*)+\d+\], )+\[(?:\d+,\s*)+\d+\]\])"
asks1 = lob.iloc[:,0].str.findall(pattern1)
asks2 = lob.iloc[:,0].str.findall(pattern2)

In [27]:
ask_list = []
for ask in asks2:
    if len(ask) == 0:
        ask_list.append('[]')
    else:
        ask_list.append(ask[0][0])

In [28]:
for i in range(len(ask_list)):
    if ask_list[i] == '[]':
        if asks1[i] != []:
            ask_list[i] = asks1[i][0]

In [29]:
lob['Asks'] = ask_list

We do the same for the bid data...

In [30]:
pattern1 = r"'bid', (\[\[(?:\d+,\s*\d+)\]\])"
pattern2 = r"'bid', (\[(\[(?:\d+,\s*)+\d+\], )+\[(?:\d+,\s*)+\d+\]\])"
bids1 = lob.iloc[:,0].str.findall(pattern1)
bids2 = lob.iloc[:,0].str.findall(pattern2)

In [31]:
bid_list = []
for bid in bids2:
    if len(bid) == 0:
        bid_list.append('[]')
    else:
        bid_list.append(bid[0][0])

In [32]:
for i in range(len(bid_list)):
    if bid_list[i] == '[]':
        if bids1[i] != []:
            bid_list[i] = bids1[i][0]

In [33]:
lob['Bids'] = bid_list

We get rid of the original column so that we are left with just our initial column. We also add in the original row (imported as a header initially).

In [34]:
lob = lob.drop(columns="[0.000, Exch0, [['bid', []], ['ask', []]]]")
new_row = pd.DataFrame({'Timestamps':'0.000', 'Bids':'[]', 'Asks':'[]'}, index=[0])
lob = pd.concat([new_row,lob.loc[:]]).reset_index(drop=True)

Now, we export the data to a cleaned csv file for re-import to an EDA procedure.

In [35]:
lob

,Timestamps,Bids,Asks
0,0.000,[],[]
1,1.922,[],"[[708, 4]]"
2,2.790,"[[5, 1]]","[[708, 4]]"
3,3.255,"[[59, 1]]","[[708, 4]]"
4,3.379,"[[59, 1]]","[[596, 4]]"
...,...,...,...
320094,30599.728,"[[105, 9], [104, 8], [101, 1], [100, 5], [89, ...","[[166, 5], [167, 1], [169, 5], [329, 2], [419,..."
320095,30599.759,"[[105, 9], [104, 8], [101, 1], [100, 5], [89, ...","[[166, 5], [167, 1], [169, 5], [419, 3], [466,..."
320096,30599.821,"[[105, 9], [104, 8], [101, 1], [100, 5], [79, ...","[[166, 5], [167, 1], [169, 5], [419, 3], [466,..."
320097,30599.914,"[[105, 9], [104, 8], [101, 1], [100, 5], [79, ...","[[165, 5], [166, 5], [167, 1], [419, 3], [466,..."


In [36]:
lob.to_csv(f'./data/cleaned_lob_data_{name}.csv')

## Tape Cleaning

In [38]:
tape = pd.read_csv('./data/UoB_Set01_2024-06-26tapes.csv', names=['Timestamp', 'Price', 'Quantity'], index_col=0)

In [39]:
tape

,Price,Quantity
Timestamp,,
7.347,108,1
12.617,107,1
15.097,105,1
15.841,104,2
16.089,105,3
...,...,...
30588.537,103,2
30592.381,108,3
30593.311,108,1


In [ ]:
tape.to_csv('cleaned_tape_data_2024-06-26.csv')